**This notebook is the PHASE 2 of the 3'READS+ pipeline, which will do the following:**
1. Update Refseq 3'UTR definition using polyA_DB3
2. Assign pA clusters identified in PHASE 1 to genomic features such as TSS upstream antisense regions, 5'UTRs, CDSs, introns, and 3'UTRs
3. Plot distribution of genomic features in each sample
4. APA (alternative polyadenylation) analysis
5. DE (differential gene expression) analysis
6. Automatic qPCR primer design for validation of APA
4. Other tasks such as calculating 5'UTR, CDS, and 3'UTR sizes and predicting 5' ORF  

**Required softwares**
 * DZ.misc.R in the modules folder 
 * R packages: foreach, doParallel, ShortRead, Biostrings, dplyr, tidyr, grid, ggplot2, ...
 * Linux environment
 
**Hardware requirements**
 * Muticore CPU for parallel computing

## Set parameters for Part 2
These parameters need to be specified for different projects and experiments. 

In [ ]:
PROJECT = 'rip' # Project name

EXPERIMENT = 'hur' # Experiment name

GENOME = 'mm9' # Genome name, currently either mm9 or hg19

ANALYSIS = 'normalize_to_IgG' # Each data set can be analyzed with different parameters
#ANALYSIS = 'normalize_to_C' # Each data set can be analyzed with different parameters
#ANALYSIS = 'AS_vs_NT' # Each data set can be analyzed with different parameters

In [4]:
if(ANALYSIS == 'normalize_to_IgG'){
  # Define which samples to focus on
  treatments = c("NT", "AS") # Order matters. Earlier ones will be denominators
  fractions = c("IgG", "HuR") # Order matters. Earlier ones will be denominators 
  comparisons = c("HuR_IgG") # The type of comparisons you want to make. "HuR_IgG": HuR vs IgG
  
  # Set up dirs
  data_dir = paste0("../../data/", experiment)
  result_dir = paste0("../../results/", experiment,"/", setting)
  system(paste0("mkdir ", result_dir))
    
  uPASS = F # Use unique PASS reads for analysis?
  if(uPASS){
    cluster_file = paste0('../../results/', experiment, '/clusters.using.unique.reads.csv')
    lowest_count = 2
    filter_strength = 0.5
  }else{
    cluster_file = paste0('../../results/', experiment, '/clusters.using.all.reads.csv')
    lowest_count = 5
    filter_strength = 0.5
  }
    
  # Settings for filtering low expression pAs 
  min_cluster_read_num = 2 # minimal reads number per cluster in the cluster file
  lowest_usage = 0 # 5% 
  lowest_rpm = 0
  USE_PSEUDOCOUNTS = F
  
  # Settings for identifying aUTRs
  neighbor = F # Do not calculate aUTRs for each neighboring isoforms
  toptwo = T # Pick top two isoforms for each gene
  
  # How to calculate 3'UTR length
  quick_UTR_length = F  # If true, the UTR length will be calculated more accurately but slower
}

if(ANALYSIS == 'normalize_to_C'){
  # Define which samples to focus on
  treatments = c("NT", "AS") # Order matters. Earlier ones will be denominators
  fractions = c("C", "IgG", "HuR") # Order matters. Earlier ones will be denominators 
  comparisons = c("IgG_C", "HuR_C") # The type of comparisons you want to make. "HuR_IgG": HuR vs IgG
  
  # Set up dirs
  data_dir = paste0("../../data/", experiment)
  result_dir = paste0("../../results/", experiment,"/", setting)
  system(paste0("mkdir ", result_dir))
    
  uPASS = F # Use unique PASS reads for analysis?
  if(uPASS){
    cluster_file = paste0('../../results/', experiment, '/clusters.using.unique.reads.csv')
    lowest_count = 2
    filter_strength = 0.5
  }else{
    cluster_file = paste0('../../results/', experiment, '/clusters.using.all.reads.csv')
    lowest_count = 5
    filter_strength = 0.5
  }
    
  # Settings for filtering low expression pAs 
  min_cluster_read_num = 2 # minimal reads number per cluster in the cluster file
  lowest_usage = 0 # 5% 
  lowest_rpm = 0
  USE_PSEUDOCOUNTS = F
  
  # Settings for identifying aUTRs
  neighbor = F # Do not calculate aUTRs for each neighboring isoforms
  toptwo = T # Pick top two isoforms for each gene
  
  # How to calculate 3'UTR length
  quick_UTR_length = F  # If true, the UTR length will be calculated more accurately but slower
}

if(ANALYSIS == 'AS_vs_NT'){
  # Define which samples to focus on
  treatments = c("NT", "AS") # Order matters. Earlier ones will be denominators
  fractions = c("C", "IgG", "HuR") # Order matters. Earlier ones will be denominators 
  comparisons = c("AS_NT") # The type of comparisons you want to make. "HuR_IgG": HuR vs IgG
  
  # Set up dirs
  data_dir = paste0("../../data/", experiment)
  result_dir = paste0("../../results/", experiment,"/", setting)
  system(paste0("mkdir ", result_dir))
    
  uPASS = F # Use unique PASS reads for analysis?
  if(uPASS){
    cluster_file = paste0('../../results/', experiment, '/clusters.using.unique.reads.csv')
    lowest_count = 2
    filter_strength = 0.5
  }else{
    cluster_file = paste0('../../results/', experiment, '/clusters.using.all.reads.csv')
    lowest_count = 5
    filter_strength = 0.5
  }
    
  # Settings for filtering low expression pAs 
  min_cluster_read_num = 2 # minimal reads number per cluster in the cluster file
  lowest_usage = 0 # 5% 
  lowest_rpm = 0
  USE_PSEUDOCOUNTS = F
  
  # Settings for identifying aUTRs
  neighbor = F # Do not calculate aUTRs for each neighboring isoforms
  toptwo = T # Pick top two isoforms for each gene
  
  # How to calculate 3'UTR length
  quick_UTR_length = F  # If true, the UTR length will be calculated more accurately but slower
}


header = strsplit(readLines(cluster_file, n=1), ",")[[1]]
sample_names = grep(paste0("(", paste0(treatments,collapse="|"), ")_(", paste0(fractions,collapse="|"), ")_\\d+"), header[-c(1:3)], value=T)

# Calculate batch numbers
batches = unique(sub(".+_(\\d+.?)$", "\\1", sample_names)) 

ERROR: Error in eval(expr, envir, enclos): object 'ANALYSIS' not found


In [3]:
oldw = getOption("warn")
options(warn = -1)

require(ggplot2)
require(gplots)
require(GGally)
require("org.Hs.eg.db")
library(GenomicFeatures)
require("BSgenome.Hsapiens.UCSC.hg19") 

options(warn = oldw)